In [29]:
import sys
!{sys.executable} -m pip install geopandas

  Using cached https://files.pythonhosted.org/packages/24/11/d77c157c16909bd77557d00798b05a5b6615ed60acb5900fbe6a65d35e93/geopandas-0.4.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/29/72/5c1888c4948a0c7b736d10e0f0f69966e7c0874a660222ed0a2c2c6daa9f/pyproj-1.9.5.1.tar.gz
  Running setup.py install for pyproj ... error
    Complete output from command /usr/bin/python3 -u -c "import setuptools, tokenize;__file__='/tmp/pip-install-0gfkqfym/pyproj/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record /tmp/pip-record-o2lds4pq/install-record.txt --single-version-externally-managed --compile:
    using bundled proj4..
    nad2bin.c: In function ‘main’:
    nad2bin.c:362:21: warning: comparison of integer expressions of different signedness: ‘size_t’ {aka ‘long unsigned int’} and ‘int’ [-Wsign-compare]
                         != 4 * ct.lim.lam )
                

Command "/usr/bin/python3 -u -c "import setuptools, tokenize;__file__='/tmp/pip-install-0gfkqfym/pyproj/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record /tmp/pip-record-o2lds4pq/install-record.txt --single-version-externally-managed --compile" failed with error code 1 in /tmp/pip-install-0gfkqfym/pyproj/


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
# import geopandas as gpd
import numpy as np
import os

In [5]:
interview_ds = pd.read_csv('../Dept_24-00013/24-00013_UOF_2008-2017_prepped.csv')
employment_ds = pd.read_csv('../Dept_24-00013/24-00013_ACS_data/24-00013_ACS_employment/ACS_15_5YR_S2301_with_ann.csv')
edu_ds = pd.read_csv('../Dept_24-00013/24-00013_ACS_data/24-00013_ACS_education-attainment/ACS_15_5YR_S1501_with_ann.csv')
edu25_ds = pd.read_csv('../Dept_24-00013/24-00013_ACS_data/24-00013_ACS_education-attainment-over-25/ACS_15_5YR_B15003_with_ann.csv')
ownerhousing_ds = pd.read_csv('../Dept_24-00013/24-00013_ACS_data/24-00013_ACS_owner-occupied-housing/ACS_15_5YR_S2502_with_ann.csv')
poverty_ds = pd.read_csv('../Dept_24-00013/24-00013_ACS_data/24-00013_ACS_poverty/ACS_15_5YR_S1701_with_ann.csv')
income_ds = pd.read_csv('../Dept_24-00013/24-00013_ACS_data/24-00013_ACS_income/ACS_15_5YR_S1903_with_ann.csv')
race_ds = pd.read_csv('../Dept_24-00013/24-00013_ACS_data/24-00013_ACS_race-sex-age/ACS_15_5YR_DP05_with_ann.csv')


In [6]:
interview_ds.head(10)

,LOCATION_LONGITUDE,LOCATION_LATITUDE,INCIDENT_UNIQUE_IDENTIFIER,INCIDENT_DATE,INCIDENT_TIME,SUBJECT_INJURY,TYPE_OF_FORCE_USED,WEAPON_OR_TOOL_USED,SUBJECT_RACE,SUBJECT_GENDER,SUBJECT_AGE,REASON_FOR_FORCE,LOCATION_DISTRICT
0,X,Y,PoliceUseOfForceID,ResponseDate,NaN,SubjectInjury,ForceTypeDetail,ForceTypeAction,Race,Sex,EventAge,TypeOfResistance,Precinct
1,-93.27169927,45.00803595,1,2008-01-01,01:15:31.000Z,-1,BodilyForceType,Push Away,Black,Female,25,Assaulted Officer,2
2,-93.27169927,45.00803595,2,2008-01-01,01:15:31.000Z,-1,BodilyForceType,Body Weight to Pin,Black,Female,25,Assaulted Officer,2
3,-93.27169927,45.00803595,3,2008-01-01,01:15:31.000Z,-1,BodilyForceType,Punches,Black,Female,25,Assaulted Officer,2
4,-93.24701977,44.95974418,4,2008-01-01,01:27:27.000Z,0,BodilyForceType,Joint Lock,White,Male,27,Commission of Crime,3
5,-93.24701977,44.95974418,5,2008-01-01,01:27:27.000Z,0,BodilyForceType,Body Weight to Pin,White,Male,27,Commission of Crime,3
6,-93.24701977,44.95974418,6,2008-01-01,01:27:27.000Z,0,TaserDeployed,Firing Darts,White,Male,27,Commission of Crime,3
7,-93.27314066,44.98080828,7,2008-01-01,01:29:47.000Z,NaN,BodilyForceType,Push Away,White,Male,39,Commission of Crime,1
8,-93.27314066,44.98080828,8,2008-01-01,01:29:47.000Z,NaN,BodilyForceType,Body Weight to Pin,White,Male,39,Commission of Crime,1
9,-93.27310907,44.97918668,9,2008-01-01,01:40:02.000Z,NaN,BodilyForceType,Joint Lock,White,Male,42,Tensed,1


In [9]:
interview_ds.shape

(25802, 13)

In [18]:
interview_ds.LOCATION_DISTRICT.unique()

array(['Precinct', '2', '3', '1', '5', '4', nan, '0'], dtype=object)

In [21]:
income_ds['GEO.display-label'].unique()

array(['Geography', 'Census Tract 1.01, Hennepin County, Minnesota',
       'Census Tract 1.02, Hennepin County, Minnesota',
       'Census Tract 3, Hennepin County, Minnesota',
       'Census Tract 6.01, Hennepin County, Minnesota',
       'Census Tract 6.03, Hennepin County, Minnesota',
       'Census Tract 11, Hennepin County, Minnesota',
       'Census Tract 17, Hennepin County, Minnesota',
       'Census Tract 22, Hennepin County, Minnesota',
       'Census Tract 24, Hennepin County, Minnesota',
       'Census Tract 27, Hennepin County, Minnesota',
       'Census Tract 32, Hennepin County, Minnesota',
       'Census Tract 33, Hennepin County, Minnesota',
       'Census Tract 38, Hennepin County, Minnesota',
       'Census Tract 59.01, Hennepin County, Minnesota',
       'Census Tract 59.02, Hennepin County, Minnesota',
       'Census Tract 68, Hennepin County, Minnesota',
       'Census Tract 77, Hennepin County, Minnesota',
       'Census Tract 78.01, Hennepin County, Minnesota',

In [24]:
poverty_ds.shape

(300, 369)

In [25]:
income_ds.shape

(300, 123)

In [7]:
income_ds.head()

,GEO.id,GEO.id2,GEO.display-label,HC01_EST_VC02,HC01_MOE_VC02,HC02_EST_VC02,HC02_MOE_VC02,HC01_EST_VC04,HC01_MOE_VC04,HC02_EST_VC04,...,HC02_EST_VC39,HC02_MOE_VC39,HC01_EST_VC40,HC01_MOE_VC40,HC02_EST_VC40,HC02_MOE_VC40,HC01_EST_VC41,HC01_MOE_VC41,HC02_EST_VC41,HC02_MOE_VC41
0,Id,Id2,Geography,Total; Estimate; Households,Total; Margin of Error; Households,Median income (dollars); Estimate; Households,Median income (dollars); Margin of Error; Hous...,Total; Estimate; Households - One race-- - White,Total; Margin of Error; Households - One race-...,Median income (dollars); Estimate; Households ...,...,Median income (dollars); Estimate; PERCENT IMP...,Median income (dollars); Margin of Error; PERC...,Total; Estimate; PERCENT IMPUTED - Family inco...,Total; Margin of Error; PERCENT IMPUTED - Fami...,Median income (dollars); Estimate; PERCENT IMP...,Median income (dollars); Margin of Error; PERC...,Total; Estimate; PERCENT IMPUTED - Nonfamily i...,Total; Margin of Error; PERCENT IMPUTED - Nonf...,Median income (dollars); Estimate; PERCENT IMP...,Median income (dollars); Margin of Error; PERC...
1,1400000US27053000101,27053000101,"Census Tract 1.01, Hennepin County, Minnesota",1111,60,59414,12451,62.8,7.9,63782,...,(X),(X),34.3,(X),(X),(X),7.2,(X),(X),(X)
2,1400000US27053000102,27053000102,"Census Tract 1.02, Hennepin County, Minnesota",1783,100,43438,3688,58.0,4.9,44491,...,(X),(X),21.6,(X),(X),(X),18.7,(X),(X),(X)
3,1400000US27053000300,27053000300,"Census Tract 3, Hennepin County, Minnesota",1352,70,65407,6571,73.5,7.0,77692,...,(X),(X),16.3,(X),(X),(X),22.2,(X),(X),(X)
4,1400000US27053000601,27053000601,"Census Tract 6.01, Hennepin County, Minnesota",1997,85,80737,12211,86.6,6.4,82105,...,(X),(X),22.4,(X),(X),(X),23.9,(X),(X),(X)


In [34]:
type(income_ds['HC01_MOE_VC40'][2])

str